### === some notes about this file === #
- Ascon family includes AEAD, hash, and mac algorithms. This file is an implementation for AEAD only
- Also, Ascon has 3 AEAD variants, this file is only for Ascon-128 variant, since it's the primary variant of Ascon
### AEAD = Authenticated Encryption with Associated Data
- Ascon submission paper: https://csrc.nist.gov/CSRC/media/Projects/lightweight-cryptography/documents/finalist-round/updated-spec-doc/ascon-spec-final.pdf

- this file is meant to be a quick hands on Ascon-128 encryption/decryption


### === Ascon core steps === #
- initialize > associated data > plaintext/ciphertext > finalization

### === Ascon parameters === #
- S = ascon state = 64 columns * 5 raws = 320 bits
- data block size = rate (r) = 1st raw of the state = 64 bits
- iv = 64 bits = 80400C0600000000
- key size = 128 bits
- nonce = 128 bits
- tag = 128 bits


In [1]:
def ascon_encrypt(key, nonce, associateddata, plaintext): # input
    # make sure parameters are within the correct ranges
    assert(len(key) == 16 and len(nonce) == 16)

    # parameters
    S = [0, 0, 0, 0, 0]    # state rows
    a = 12  # initial & final rounds
    b = 6   # intermediate rounds
    rate = 8    # bytes

    # process
    ascon_initialize(S, a, key, nonce)
    ascon_process_associated_data(S, b, rate, associateddata)
    ciphertext = ascon_process_plaintext(S, b, rate, plaintext)
    tag = ascon_finalize(S, a, key)

    #output = ciphertext (same size as plaintext) + tag (128-bits)
    return ciphertext + tag

- The ascon_encrypt function takes the encryption parameters as input: key, nonce, associateddata, and plaintext.
- It performs a simple assertion to ensure that the lengths of key and nonce are both 16 bytes (128 bits), as required.

### Defines the parameters for the Ascon algorithm:
 - S: The state, initialized as a list of zeros with 5 elements.
 - a: The number of initial and final rounds.
 - b: The number of intermediate rounds.
 - rate: The rate in bytes (size of a data block).
 
 ### Calls several helper functions to perform the Ascon encryption process:
 - ascon_initialize: Initializes the Ascon state with the provided key and nonce.
 - ascon_process_associated_data: Processes the associated data (if any).
 - ascon_process_plaintext: Processes the plaintext data.
 - ascon_finalize: Finalizes the encryption process and generates the tag.
 
 ### Returns the ciphertext concatenated with the authentication tag.
- The ascon_decrypt function follows a similar structure, but it's for decryption and includes additional steps for tag verification.

- The remaining code provides helper functions and explanations for understanding the Ascon algorithm's internals, including the substitution layer (S-box) implementation using both a lookup table (LUT) and a logic gate-based approach.

In [2]:
def ascon_decrypt(key, nonce, associateddata, ciphertext): # input
    # make sure parameters are within the correct ranges
    assert(len(key) == 16 and len(nonce) == 16 and len(ciphertext) >= 16)

    # parameters
    S = [0, 0, 0, 0, 0]    # state raws
    a = 12  # inititial & final rounds
    b = 6   # intermediate rounds
    rate = 8    # bytes

    # process
    ascon_initialize(S, a, key, nonce)
    ascon_process_associated_data(S, b, rate, associateddata)
    plaintext = ascon_process_ciphertext(S, b, rate, ciphertext[:-16])  # ignore the tag (last 16 bytes)
    tag = ascon_finalize(S, a, key)

    # output
    if tag == ciphertext[-16:]: # check the tag for authentication (last 16 bytes)
        return plaintext
    else:
        return None

### Function: ascon_decrypt
- This function is responsible for decrypting ciphertext using the Ascon-128 encryption algorithm. Here's a breakdown:

### Parameters:

 - key: The encryption key (128 bits).
 - nonce: Nonce used during encryption (128 bits).
 - associateddata: Associated data used during encryption.
 - ciphertext: The encrypted ciphertext.

### Parameter Check:
- It checks if the length of the key, nonce, and ciphertext are valid.

### Initialization:
- Initializes the state S, number of initial and final rounds a, number of intermediate rounds b, and the rate rate.

### Process:
- Initializes the state using the ascon_initialize function.
- Processes the associated data using the ascon_process_associated_data function.
- Decrypts the ciphertext using the ascon_process_ciphertext function, excluding the tag (last 16 bytes).
- Finalizes the state using the ascon_finalize function to obtain the tag.

### Output:
- Checks if the obtained tag matches the tag from the ciphertext for authentication. If they match, returns the decrypted plaintext; otherwise, returns None.

In [3]:
def ascon_initialize(S, a, key, nonce):
    iv = bytes.fromhex('80400c0600000000')
    '''
    '80400c0600000000' is a fixed iv for Ascon-128, representing its main paramters as following:
      # 80 -> 128 -> key length (k)
      # 40 -> 64 -> rate (r)
      # 0c -> 12 -> a -> number of initial and final permuations
      # 06 -> 6 -> b -> number of intermediate permutaions
      # 0000000 = padding
    '''

    initial_sate = iv + key + nonce # initial state = iv (64 bits) + key (128 bits) + nonce (128 bits) = 320 bits = 40 bytes

    '''
    # filling the initial state block as follows (in decimal):
      1st raw S[0] = iv
      2nd & 3rd raws S[1], S[2] = key
      4rd & 5th raws S[4], S[5] = nonce
    '''
    S[0], S[1], S[2], S[3], S[4] = bytes_to_state(initial_sate)

    # initial permutation of the state
    ascon_permutation(S, a)

    # zero_key = key padded with 0s put before it
    # 0*||K = 24 of zero bytes + 16-byte key = 40 bytes total = 320 bits
    # initialize the zero_key and put it in a block
    zero_key = bytes_to_state(b"\x00" * (40-len(key)) + key)

    # XOR the state with the zero_key
    for i in range(5): S[i] ^= zero_key[i]

### Function: ascon_initialize
This function initializes the state S using the provided key, nonce, and a fixed initialization vector (IV). Here's an explanation:

Initialization Vector (IV):

A fixed IV specific to Ascon-128 is used ('80400c0600000000') to represent its main parameters: key length, rate, number of permutations, etc.
Initial State:

Constructs the initial state block by concatenating the IV, key, and nonce.
Fills the state with specific values from the initial state block.
Initial Permutation:

Applies the initial permutation to the state using the ascon_permutation function.
Zero Key:

Prepares a zero-padded key for XOR operation with the state.
XOR Operation:

Performs an XOR operation between the state and the zero-padded key.
This initialization process sets up the state for subsequent encryption or decryption operations in Ascon-128.

In [4]:
# ad = associated data
def ascon_process_associated_data(S, b, rate, associateddata):
    if len(associateddata) > 0:
        # == padding == #
        # associated data is padded by 1 followed by 0s --> 1 || 0s

        # length of last block in the raw associated data (before padding)
        ad_lastlen = len(associateddata) % rate

        # calculate how many zero bytes needed for padding
        ad_zero_bytes = rate - (ad_lastlen % rate) -  1

        # keep in mind that 0x80 = 128 = 10000000 in binary
        ad_padding = bytes([0x80] + [0x00]*ad_zero_bytes)
        ad_padded = associateddata + ad_padding

        # == absorbtion of associated data ==#
        # xor padded associated data with the rate, then permute
        for block in range(0, len(ad_padded), rate):
            S[0] ^= int(ad_padded[block:block+8].hex(), 16)
            ascon_permutation(S, b)

    # state is xored with 1 for domain separation --> S ^ (0**319 || 1)
    # we only need to xor 1 with the last raw because first 4 raws will remain unchanged
    S[4] ^= 1



### This function ascon_process_associated_data is responsible for processing the associated data in the Ascon encryption algorithm. Here's a breakdown of the code:
- This function takes four parameters: S (state array), b (number of intermediate rounds), rate (rate of absorption in bytes), and associateddata (associated data to be processed).

- It first checks if the length of the associated data is greater than 0, ensuring there is data to process.
- The associated data is padded according to the rate. It starts with appending a byte with hexadecimal value 0x80 (binary 10000000), followed by zero bytes to fill the block.

- The calculation ensures that the padding starts with 0x80 followed by enough zeros to complete the block, considering the length of the associated data.

- The function then absorbs the padded associated data into the state array S. It does so by iterating over blocks of data (with size equal to the rate) and XORing each block with the state, followed by permutation using the ascon_permutation function.

- Finally, the state is XORed with 1 for domain separation. Since only the last row of the state requires this XOR operation, it's performed only on S[4].

In [5]:
### ============================================================================== ###

def ascon_process_plaintext(S, b, rate, plaintext):
    # == padding == #
    # plaintext is padded by 1 followed by 0s --> 1 || 0s
    # note: we need padding only to be able to calculate the the new rate, otherwise the ciphertext is truncated eventually and the padding is discarded

    # length of last block in the raw plaintext (before padding)
    p_lastlen = len(plaintext) % rate

    # calculate how many zero bytes needed for padding
    p_zero_bytes = (rate - p_lastlen) - 1

    # keep in mind that 0x80 = 128 = 10000000 in binary
    p_padding = bytes([0x80] + [0x00]*p_zero_bytes)
    p_padded = plaintext + p_padding

    # == absorbtion of plaintext & squeezing of ciphertext == #
    # processing of first t-1 blocks (all blocks except the last one)
    ciphertext = bytes([])
    blocks = len(p_padded) - rate # length of plaintext blocks except the last block

    for block in range(0, blocks, rate): # ex: if len(p_padded)=24, p1 = 0 to 8, p2 = 8 to 16,, ignoring the last block which is 16 to 24
        S[0] ^= int(p_padded[block:block+8].hex(), 16)  # absorbing = xoring plaintext with the rate
        ciphertext += S[0].to_bytes(8, 'big') # squeezing
        ascon_permutation(S, b)

    #  processing of last block
    p_last = int(p_padded[blocks:].hex(), 16)
    S[0] ^= p_last
    # there is no intermediate permutation after the last block

    # last block of ciphertext is truncated to become with the same length of the last block of raw plaintext (before padding)
    # intended result: len(ciphertext) == len(plaintext)
    ciphertext += S[0].to_bytes(8, 'big')[:p_lastlen]
    return ciphertext



### This function ascon_process_plaintext is responsible for processing the plaintext in the Ascon encryption algorithm. Here's a breakdown of the code:

- The function starts by padding the plaintext to ensure its length is a multiple of the rate. This padding is done by appending a byte with hexadecimal value 0x80 (binary 10000000), followed by zero bytes to fill the block.

- The length of the last block in the plaintext (before padding) is calculated to determine the padding required.

- The number of zero bytes needed for padding is calculated to ensure that the total length becomes a multiple of the rate.

- The function then processes the plaintext blocks by XORing them with the state array S and squeezing the result into the ciphertext. This process iterates over all blocks except the last one.

- Each block of plaintext is absorbed into the state by XORing it with the rate, and then the state is permuted using the ascon_permutation function.

- Finally, the last block of plaintext is processed similarly, but without intermediate permutation. The resulting ciphertext is truncated to have the same length as the last block of the original plaintext before padding.

In [6]:
### ============================================================================== ###

def ascon_process_ciphertext(S, b, rate, ciphertext):
    # == padding == #
    # ciphertext is padded by 1 followed by 0s --> 1 || 0s

    # length of last block of ciphertext
    c_lastlen = len(ciphertext) % rate

    # calculate how many zero bytes needed for padding
    c_zero_bytes = (rate - c_lastlen) - 1

    # keep in mind that 0x80 = 128 = 10000000 in binary
    c_padding = bytes([0x80] + (c_zero_bytes)*[0x00])
    c_padded = ciphertext + c_padding

    # == absorbtion of ciphertext & squeezing of plaintext == #
    # processing of first t-1 blocks  (all blocks except the last one)
    plaintext = bytes([])
    blocks = len(c_padded) - rate # length of ciphetext blocks except the last block

    for block in range(0, blocks, rate):
        Ci = int(c_padded[block:block+8].hex(), 16) # 1 byte block of ciphertext
        plaintext += (S[0] ^ Ci).to_bytes(8, 'big')
        S[0] = Ci # rate will become the ciphertext block
        ascon_permutation(S, b)

    # processing of last block t
    c_last = int(c_padded[blocks:].hex(), 16) # last block
    plaintext += (c_last ^ S[0]).to_bytes(8, 'big')[:c_lastlen]

    # rate = S[0] ^ (plaintext || 1 || 0s)
    padded_plaintext = int((plaintext[blocks:] + c_padding).hex(), 16)
    S[0] ^= padded_plaintext

    return plaintext



### This function ascon_process_ciphertext is responsible for processing the ciphertext in the Ascon encryption algorithm. Here's a breakdown of the code:

- The function starts by padding the ciphertext to ensure its length is a multiple of the rate. Similar to plaintext padding, this is done by appending a byte with hexadecimal value 0x80 (binary 10000000), followed by zero bytes to fill the block.

- The length of the last block in the ciphertext (before padding) is calculated to determine the padding required.

- The number of zero bytes needed for padding is calculated to ensure that the total length becomes a multiple of the rate.

- The function then processes the ciphertext blocks by XORing them with the state array S and squeezing the result into the plaintext. This process iterates over all blocks except the last one.

- Each block of ciphertext is XORed with the state and added to the plaintext.

- Finally, the last block of ciphertext is processed similarly to previous blocks, and the resulting plaintext is appended.

- The function then updates the state with the last block of ciphertext and calculates the XOR with the padded plaintext to update the rate.

In [7]:
### ============================================================================== ###

def ascon_finalize(S, a, key):
    assert(len(key)) == 16

    # == step 1 == #
    # key is padded with 8 bytes of 0s before it & 16 bytes of 0s after it (0*8 || K || 0*16),
    # then it's xored with the state
    # since, only the 2nd & 3rd raw of the state will take effect, we only xor them without implementing the padding step
    S[1] ^= int(key[:8].hex(), 16)
    S[2] ^= int(key[8:].hex(), 16)

    ascon_permutation(S, a)

    # == step 2 ==#
    # 4th & 5th raws of the state are xored with the key, and the result will be the tag
    S[3] ^= int(key[:8].hex(), 16)
    S[4] ^= int(key[8:].hex(), 16)
    tag = (S[3].to_bytes(8, 'big') + S[4].to_bytes(8, 'big'))

    return tag



### This function ascon_finalize is responsible for finalizing the Ascon encryption process by generating the authentication tag. Here's a breakdown of the code:

- The function starts by ensuring that the length of the key is 16 bytes (128 bits) as required for Ascon encryption.

- Then, it pads the key with 8 bytes of zero before it and 16 bytes of zero after it to make a total of 40 bytes (320 bits). This padded key is XORed with the state array S, specifically with the 2nd and 3rd rows of the state.

- After XORing the key with the state, the function performs the initial permutation of the state using the ascon_permutation function.

- Next, the function XORs the 4th and 5th rows of the state with the key. The resulting values from these XOR operations constitute the authentication tag.

- Finally, the function returns the tag, which consists of the 4th and 5th rows of the state converted to bytes and concatenated together. This tag serves as a means of verifying the integrity and authenticity of the encrypted data.

In [8]:
### ============================================================================== ###
# === ascon permutation === #

def ascon_permutation(S, rounds):
    assert(rounds <= 12)

    for r in range(12-rounds, 12):
        # --- step 1: add round constants --- #
        S[2] ^= (0xf0 - r*0x10 + r*0x1)

        # --- step 2: substitution layer --- #
        # see sbox instructions at: https://ascon.iaik.tugraz.at/images/sbox_instructions.c
        S[0] ^= S[4]
        S[4] ^= S[3]
        S[2] ^= S[1]

        # this line summarizes the NOR & ANDing operations for all raws
        T = [(S[i] ^ 0xFFFFFFFFFFFFFFFF) & S[(i+1)%5] for i in range(5)]

        for i in range(5):
            S[i] ^= T[(i+1)%5]
        S[1] ^= S[0]
        S[0] ^= S[4]
        S[3] ^= S[2]
        S[2] ^= 0XFFFFFFFFFFFFFFFF # XORing with 1s = NOT operation

        # --- step 3: linear diffusion layer --- #
        S[0] ^= rotr(S[0], 19) ^ rotr(S[0], 28)
        S[1] ^= rotr(S[1], 61) ^ rotr(S[1], 39)
        S[2] ^= rotr(S[2],  1) ^ rotr(S[2],  6)
        S[3] ^= rotr(S[3], 10) ^ rotr(S[3], 17)
        S[4] ^= rotr(S[4],  7) ^ rotr(S[4], 41)



### This function ascon_permutation implements the permutation step of the Ascon encryption algorithm. It consists of three main steps: adding round constants, a substitution layer, and a linear diffusion layer. Here's a detailed explanation of each step:

- **Step 1 (Add Round Constants):** In this step, round constants are added to the 3rd row of the state array S. These constants are derived based on the current round number r. The specific calculation (0xf0 - r*0x10 + r*0x1) generates round constants for each round.

- **Step 2 (Substitution Layer):** This step involves several bitwise operations and XOR operations to achieve a nonlinear mixing of the state. Each row of the state is modified based on the values of other rows.

- **Step 3 (Linear Diffusion Layer):** This step introduces diffusion by applying a series of bitwise rotations (rotations to the right) on each row of the state. Different rotation constants are applied to different rows.

In [9]:
### ============================================================================== ###
# === helper functions === #

def bytes_to_state(bytes):
    bytes = bytes.hex()
    return [int(bytes[16*w:16*(w+1)], 16) for w in range(5)]

def rotr(val, r):
    return (val >> r) | ((val & (1<<r)-1) << (64-r))

def get_random_bytes(num):
    from os import urandom
    return (urandom(num))

def hex_print(data):
    for text, val in data:
        print("{text}:{align}0x{val}".format(text=text, val=val.hex(), align=(19-len(text))*" "))



### let's go through each helper function:

1. `bytes_to_state(bytes)`: This function converts a byte array into a state array. It first converts the byte array into a hexadecimal string using the `hex()` method. Then, it divides this string into 16-character chunks and converts each chunk into an integer using base 16 (hexadecimal). Finally, it returns a list containing these integers, forming the state array.

2. `rotr(val, r)`: This function performs a right rotation (circular shift) of the value `val` by `r` bits. It achieves this by first shifting `val` to the right by `r` bits using the right shift operator `>>`. Then, it extracts the bits that were shifted out (which are `r` rightmost bits of `val`) using bitwise AND with `(1<<r)-1`. After that, it shifts these extracted bits to the left by `(64-r)` bits and combines them with the shifted `val` using bitwise OR `|`. This effectively rotates the bits of `val` to the right by `r` positions.

3. `get_random_bytes(num)`: This function generates a byte array containing `num` random bytes using the `urandom()` function from the `os` module. It imports the `urandom()` function and calls it with the `num` parameter to obtain a byte array of random bytes, which it then returns.

4. `hex_print(data)`: This function prints data in a formatted way, where each element of `data` is a tuple containing a text label and a corresponding byte array value. It iterates over each tuple in `data`, formatting and printing the label along with the hexadecimal representation of the byte array value. It ensures that the hexadecimal representation is prefixed with `0x` and aligns the output for better readability.

These helper functions are useful for various tasks such as converting between different data representations, performing bitwise operations, generating random bytes, and printing data in a formatted manner. They provide essential functionalities that are commonly used in cryptography and other programming tasks.

In [10]:
### ============================================================================== ###
# === demo aead === #

def demo_aead():
    # demo = "=== demo encryption/decryption using Ascon-128 ==="

    associateddata = b'I am a student'
    plaintext = b'My name is Reetu Shrivastava'


    key = get_random_bytes(16) # ex: b"\xea\xa9\x11\x9a\xa3\xa9\xbd^P\xbc\xcd\xa4\xe1=\x1c\x03"
    nonce = get_random_bytes(16) # ex: b"\x1ae'\xa3fE\xdd\xb9I\x06q\xdc]\x1e\x1e\xbb"



    ciphertext = ascon_encrypt(key, nonce, associateddata, plaintext)
    tag = ciphertext[-16:]


    receivedplaintext = ascon_decrypt(key, nonce, associateddata, ciphertext)

    if receivedplaintext == None: print("verification failed!") | exit()

    # print(demo)
    print(f"associated data:    {associateddata}")
    print(f"plaintext:          {plaintext}")


    hex_print([("key", key),
                ("nonce", nonce),
                ("ciphertext", ciphertext[:-16]),
                ("tag", tag),
                ])



    print(f"received plaintext: {receivedplaintext}")

### This code snippet demonstrates the encryption and decryption process using the Ascon-128 algorithm. Here's an explanation of each part:

1. `demo_aead()`: This function is the main entry point for the demonstration. It initializes the associated data, plaintext, key, and nonce. Then, it encrypts the plaintext using the `ascon_encrypt()` function, which returns the ciphertext along with the authentication tag. After that, it attempts to decrypt the ciphertext using the `ascon_decrypt()` function and verifies the authenticity of the received plaintext.

2. `associateddata` and `plaintext`: These variables store the associated data and plaintext that will be encrypted and decrypted, respectively.

3. `key` and `nonce`: These variables store the randomly generated encryption key and nonce, which are essential for the encryption and decryption process.

4. `ciphertext = ascon_encrypt(key, nonce, associateddata, plaintext)`: This line encrypts the plaintext using the Ascon-128 encryption algorithm. It calls the `ascon_encrypt()` function with the encryption key, nonce, associated data, and plaintext as parameters. The function returns the ciphertext concatenated with the authentication tag.

5. `tag = ciphertext[-16:]`: This line extracts the last 16 bytes from the ciphertext, which represent the authentication tag.

6. `receivedplaintext = ascon_decrypt(key, nonce, associateddata, ciphertext)`: This line attempts to decrypt the ciphertext using the Ascon-128 decryption algorithm. It calls the `ascon_decrypt()` function with the encryption key, nonce, associated data, and ciphertext as parameters. If the decryption is successful and the plaintext is authentic, it returns the decrypted plaintext; otherwise, it returns `None`.

7. `print("verification failed!") | exit()`: This line is a logical OR operation (`|`) followed by the `exit()` function. However, it seems to have a typo because the logical OR operation `|` doesn't work as expected in this context. It should be replaced with `or` to function as a conditional check. If the received plaintext is `None`, indicating verification failure, the program exits.

8. `print(f"associated data:    {associateddata}")`, `print(f"plaintext:          {plaintext}")`, `print(f"received plaintext: {receivedplaintext}")`: These lines print out the associated data, plaintext, and received plaintext, respectively, for demonstration purposes.

9. `hex_print([...])`: This line prints out various information in a formatted way, including the encryption key, nonce, ciphertext (excluding the tag), and authentication tag.

Overall, this code snippet demonstrates the encryption and decryption process using Ascon-128 and verifies the authenticity of the decrypted plaintext.

In [11]:
### ============================================================================== ###
# === demo test === #
if __name__ == "__main__":
    demo_aead()

associated data:    b'I am a student'
plaintext:          b'My name is Reetu Shrivastava'
key:                0xc873926c5900c9be0f356116c3c37bcf
nonce:              0xd20d0bad0bca85ef5192f84918d827e4
ciphertext:         0xa4c181d6daaecada821ffeb5ff2b80d008cdc55dbfcc282b65acf514
tag:                0x2e039e32de518fa771e6c03828e27222
received plaintext: b'My name is Reetu Shrivastava'


In [12]:
def test_ascon_cipher():
    # Test parameters
    key = get_random_bytes(16)
    nonce = get_random_bytes(16)
    associateddata = b'I am a student'
    plaintext = b'My name is Reetu Shrivastava'
    print("\n",associateddata,"\n",plaintext)

    # Encrypt
    ciphertext = ascon_encrypt(key, nonce, associateddata, plaintext)
    print("\nCiphertext:\t",ciphertext)

    # Decrypt
    decrypted_plaintext = ascon_decrypt(key, nonce, associateddata, ciphertext)
    print("\n",decrypted_plaintext)

    # Verify correctness
    assert decrypted_plaintext == plaintext, "Encryption and decryption failed!"

    print("Encryption and decryption successful!")


if __name__ == "__main__":
    test_ascon_cipher()



 b'I am a student' 
 b'My name is Reetu Shrivastava'

Ciphertext:	 b'\xa1E\xff\xb4H\xdc`\xd7G<_\xba\x8d(zw(DD:7imsD+\xfaUQ /^\xd5\x03O\xd1s\xf9\xb2?\x1f\xd6\x85J'

 b'My name is Reetu Shrivastava'
Encryption and decryption successful!


### This code defines a function `test_ascon_cipher()` which tests the encryption and decryption functionalities of the Ascon-128 algorithm. Here's an explanation of each part:

- `test_ascon_cipher()`: This function is designed to test the encryption and decryption process using Ascon-128. It sets up test parameters including a random encryption key, a random nonce, associated data, and plaintext.

- `key`, `nonce`, `associateddata`, `plaintext`: These variables store the test parameters. The key and nonce are generated randomly using the `get_random_bytes()` function, while the associated data and plaintext are predefined.

- `print("\n",associateddata,"\n",plaintext)`: This line prints out the associated data and plaintext for demonstration purposes.

- `ciphertext = ascon_encrypt(key, nonce, associateddata, plaintext)`: This line encrypts the plaintext using the Ascon-128 encryption algorithm. It calls the `ascon_encrypt()` function with the encryption key, nonce, associated data, and plaintext as parameters, resulting in the ciphertext.

- `print("\nCiphertext:\t",ciphertext)`: This line prints out the generated ciphertext.

- `decrypted_plaintext = ascon_decrypt(key, nonce, associateddata, ciphertext)`: This line attempts to decrypt the ciphertext using the Ascon-128 decryption algorithm. It calls the `ascon_decrypt()` function with the encryption key, nonce, associated data, and ciphertext as parameters, resulting in the decrypted plaintext.

- `print("\n",decrypted_plaintext)`: This line prints out the decrypted plaintext.

- `assert decrypted_plaintext == plaintext, "Encryption and decryption failed!"`: This line asserts that the decrypted plaintext matches the original plaintext. If the assertion fails, it indicates that the encryption and decryption process did not produce the expected result.

- `print("Encryption and decryption successful!")`: If the encryption and decryption process is successful, this line prints out a success message.

- `if __name__ == "__main__":`: This conditional statement ensures that the `test_ascon_cipher()` function is executed only if the script is run directly, not if it is imported as a module into another script.

Overall, this code snippet tests the encryption and decryption functionalities of the Ascon-128 algorithm with predefined test parameters and verifies the correctness of the process.

In [13]:
def proof_of_correctness():
    # Generate random key, nonce, associated data, and plaintext
    key = get_random_bytes(16)
    nonce = get_random_bytes(16)
    associateddata = get_random_bytes(16)
    plaintext = get_random_bytes(32)

    # Encrypt the plaintext
    ciphertext = ascon_encrypt(key, nonce, associateddata, plaintext)

    # Decrypt the ciphertext
    decrypted_plaintext = ascon_decrypt(key, nonce, associateddata, ciphertext)

    # Check if decryption was successful and plaintext matches the original plaintext
    if decrypted_plaintext is None:
        print("Decryption failed. Proof of correctness failed.")
        return False
    elif decrypted_plaintext == plaintext:
        print("Decryption successful. Proof of correctness passed.")
        return True
    else:
        print("Decrypted plaintext does not match original plaintext. Proof of correctness failed.")
        return False

# Perform proof of correctness
proof_of_correctness()


Decryption successful. Proof of correctness passed.


True

### This code defines a function `proof_of_correctness()` that aims to demonstrate the correctness of the Ascon-128 encryption and decryption algorithms. Here's how the function works:

-  `key`, `nonce`, `associateddata`, `plaintext`: These variables are generated randomly to represent the key, nonce, associated data, and plaintext.

- `ciphertext = ascon_encrypt(key, nonce, associateddata, plaintext)`: The function encrypts the plaintext using Ascon-128 encryption algorithm with the randomly generated key, nonce, and associated data. The result is the ciphertext.

- `decrypted_plaintext = ascon_decrypt(key, nonce, associateddata, ciphertext)`: The function then attempts to decrypt the generated ciphertext using Ascon-128 decryption algorithm with the same key, nonce, and associated data used for encryption.

- The function checks whether the decryption was successful and whether the decrypted plaintext matches the original plaintext.

   - If decryption fails (i.e., the `ascon_decrypt` function returns `None`), it prints "Decryption failed. Proof of correctness failed." and returns `False`.
   
   - If the decrypted plaintext matches the original plaintext, it prints "Decryption successful. Proof of correctness passed." and returns `True`.
   
   - If the decrypted plaintext does not match the original plaintext, it prints "Decrypted plaintext does not match original plaintext. Proof of correctness failed." and returns `False`.

5. `proof_of_correctness()`: This line calls the `proof_of_correctness()` function to perform the proof of correctness test.
